In [9]:
# Lab 11 MNIST and Deep learning CNN
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [11]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [12]:
# MNIST dataset
mnist_train = dsets.MNIST(root='/Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='/Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

9913344it [00:01, 7193845.55it/s]                             


Extracting /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw



29696it [00:00, 6374964.25it/s]          

Extracting /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw



1649664it [00:00, 5037804.19it/s]                             


Extracting /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw



5120it [00:00, 25353998.21it/s]         

Extracting /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/seunguk/OneDrive - 중앙대학교/Programming/PyTorch/MNIST_data/MNIST/MNIST/raw



In [13]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [14]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [15]:
# instantiate CNN model
model = CNN().to(device)

In [16]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.192783207
[Epoch:    2] cost = 0.0549001992
[Epoch:    3] cost = 0.037535727
[Epoch:    4] cost = 0.030095106
[Epoch:    5] cost = 0.0249762814
[Epoch:    6] cost = 0.0206913333
[Epoch:    7] cost = 0.0169599149
[Epoch:    8] cost = 0.0149919167
[Epoch:    9] cost = 0.0135905771
[Epoch:   10] cost = 0.0110302772
[Epoch:   11] cost = 0.010470273
[Epoch:   12] cost = 0.0104556391
[Epoch:   13] cost = 0.0085240975
[Epoch:   14] cost = 0.00716155116
[Epoch:   15] cost = 0.00854859501
Learning Finished!


In [18]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/opt/homebrew/anaconda3/envs/torch_study/lib/python3.8/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/homebrew/anaconda3/envs/torch_study/lib/python3.8/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9902999997138977
